In [32]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine,MetaData
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [33]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [34]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [35]:
# Create Measurement Class
class Measurements(Base):
    __tablename__ = 'Measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(20))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)

In [36]:
# Create Measurement Class
class Stations(Base):
    __tablename__ = 'Station'
    id = Column(Integer, primary_key=True)
    station = Column(String(20))
    name = Column(String(50))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [37]:
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [38]:
# Load the cleaned csv file into a pandas dataframe
df_of_measurements = pd.read_csv('measurements_clean_final.csv')

In [39]:
# Use Orient='records' to create a list of data to write
measurements_data = df_of_measurements.to_dict(orient='records')

In [40]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [41]:
# Save the reference to the `demographics` table as a variable called `table`
meaurements_table = sqlalchemy.Table('Measurements', metadata, autoload=True)

In [42]:
conn.execute(Measurements.__table__.delete())
conn.execute(Measurements.__table__.insert(), measurements_data)

In [43]:
# Use `table.insert()` to insert the data into the table
conn.execute(Stations.__table__.delete())
conn.execute(Stations.__table__.insert(), measurements_data)

In [44]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [45]:
conn.execute("select * from Stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]